In [ ]:
import pandas as pd

In [3]:
# load the csv 
df = pd.read_csv('mpesa_transactions.csv')
df.head()


,Receipt No.,Completion Time,Details,Paid In,Withdrawn,Balance,month_name,day_name,Hour,Transaction_Type
0,TJIQI7QX9T,2025-10-18 20:26:59,Merchant Payment Fuliza M-Pesa Online to 76475...,0.0,500.0,0.0,October,Saturday,20,Till No
1,TJIQI7QX9T,2025-10-18 20:26:59,OverDraft of Credit Party,500.0,0.0,500.0,October,Saturday,20,Fuliza Loan
2,TJIQI7QSUQ,2025-10-18 20:26:27,Business Payment from 501901 - KCB 1 via API. ...,500.0,0.0,500.0,October,Saturday,20,Bank Transfer
3,TJIQI7QSUR,2025-10-18 20:26:27,OD Loan Repayment to 232323 - M-PESA Overdraw,0.0,500.0,0.0,October,Saturday,20,Fuliza Loan Repayment
4,TJIQI7QKSS,2025-10-18 19:08:16,Pay Bill Online Fuliza M-Pesa to 4097371 - 1XB...,0.0,500.0,0.0,October,Saturday,19,Pay Bill


In [27]:
def get_paybill_and_till_numbers(data: pd.DataFrame, details_col: str):
    
    if data is None or data.empty:
        return 
    
    # Define regex pattern to extract the number that appears after 'to' and before '-'
    pattern = r'to (\d+) -'

    # Apply regex to extract numbers from the specified column
    data["numbers"] = data[details_col].apply(
        lambda x: re.search(pattern, str(x).strip()).group(1) if re.search(pattern, str(x).strip()) else None
    )

    return data
    

In [4]:
df.dtypes


Receipt No.          object
Completion Time      object
Details              object
Paid In             float64
Withdrawn           float64
Balance             float64
month_name           object
day_name             object
Hour                  int64
Transaction_Type     object
dtype: object

In [8]:
# monthly income 
monthly_income = df.groupby('month_name')['Paid In'].sum()
print(monthly_income)

# standard deviation OF monthly_income
std_dev = monthly_income.std()

# mean OF monthly_income
mean = monthly_income.mean()

# income stability formula
print(f"This is the mean: {mean}")
print(f"This is the standard deviation: {std_dev}")

month_name
August        77714.07
July          62977.99
October      166609.54
September    704915.61
Name: Paid In, dtype: float64
This is the mean: 253054.3025
This is the standard deviation: 304699.05365023296


In [12]:
# Ensure score stays between 0 and 1
stability_score = max(0, 1 - (std_dev / mean))

print(stability_score)


0


In [18]:
def income_stability (df,month,paid_in):
    # monthly income 
    monthly_income = df.groupby(month)[paid_in].sum()
    # standard deviation OF monthly_income
    std_dev = monthly_income.std()

        # mean OF monthly_income
    mean = monthly_income.mean()
    # Ensure score stays between 0 and 1
    stability_score = max(0, 1 - (std_dev / mean))

    print(stability_score)


In [19]:
income_stability(df,"month_name","Paid In")

0


In [20]:
# monthly income 
monthly_income = df.groupby('Transaction_Type')['Withdrawn'].sum()

print(monthly_income)


Transaction_Type
Airtime Purchase             50.00
Bank Transfer                 0.00
Cash Withdrawal            6800.00
Fuliza Airtime             3478.00
Fuliza Loan                3701.00
Fuliza Loan Repayment     65785.07
Mshwari Deposit           13000.00
Mshwari Withdraw              0.00
Other                     66832.05
Pay Bill                 619836.65
Pochi                     14110.00
Received Money                0.00
Send Money               192738.00
Till No                   30748.00
safaricom bundles          2592.00
Name: Withdrawn, dtype: float64


### Expense Ratio

In [24]:
# expense ratio
send_money_df = df[df['Transaction_Type'] == 'Send Money']
total_sent = sum(send_money_df['Withdrawn'])
total_recieved =sum(df['Paid In'])
expense_ratio = total_sent/total_recieved
print(total_sent)
print(total_recieved)
print(expense_ratio)


192738.0
1012217.21
0.19041170027132814


### Cash Withdrawals

In [26]:
# expense ratio
withdraw_df = df[df['Transaction_Type'] == 'Cash Withdrawal']
total_withdrawals= sum(withdraw_df['Withdrawn'])
total_sent = sum(send_money_df['Withdrawn'])
cash_withdrawals = total_withdrawals/total_sent
print(total_withdrawals)
print(total_sent)
print(cash_withdrawals)


6800.0
192738.0
0.03528105511108344


### Merchant Diversity